In [ ]:
import numpy as np
import math
import re
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def data_analysis(data_dir):
    f = open(data_dir,'r')
    #out = open('output.txt','w')
    lines = f.readlines()
    throughput = []
    for line in lines:
        if "Total throughput:" in line:
            tpt = re.findall("\d+\.\d+", line)
            tpts = re.findall("\d+", line)
            tpt.append(tpts[0])
            #print(tpt)
            throughput.append(float(tpt[0]))
            #out.write(line)

    #print(throughput)
    return throughput

In [ ]:
def print_result(bianchi_result):
    print("{\"HeMcs3\", {")
    for  i in range (len(bianchi_result)):
        print("{%d, %.4f}," % (5*(i+1), bianchi_result[i]))
    print("}},")

def str_result(bianchi_result, mcs):
    str_bianchi = '{' + '\"HeMcs{:d}\"'.format(mcs) + ', {\n'
    for  i in range (len(bianchi_result)):
        str_tmp = '\t\t{' + '{:d}, {:.4f}'.format(5*(i+1), bianchi_result[i]) +'},\n'
        str_bianchi  = str_bianchi + str_tmp
    str_bianchi = str_bianchi + "}},\n"
    print(str_bianchi)
    return str_bianchi

In [ ]:
def bianchi_ax(data_rate, ack_rate, k, difs):
    # Parameters for 11ax
    nA = np.linspace(5, 50, 10)
    CWmin = 15 
    CWmax = 1023 
    L_DATA = 1500 * 8 # data size in bits
    L_ACK = 14 * 8    # ACK size in bits
    B = 1/(CWmin+1) 
    B = 0 
    EP = L_DATA/(1-B)
    T_GI = 800e-9                  # guard interval in seconds
    T_SYMBOL_ACK = 4e-6            # symbol duration in seconds (for ACK)
    T_SYMBOL_DATA = 12.8e-6 + T_GI # symbol duration in seconds (for DATA)
    T_PHY_ACK = 20e-6              # PHY preamble & header duration in seconds (for ACK)
    T_PHY_DATA = 44e-6             # PHY preamble & header duration in seconds (for DATA)
    L_SERVICE = 16                 # service field length in bits
    L_TAIL = 6                     # tail lengthh in bits
    L_MAC = (30) * 8               # MAC header size in bits
    L_APP_HDR = 8 * 8              # bits added by the upper layer(s)
    T_SIFS = 16e-6 
    T_DIFS = 34e-6 
    T_SLOT = 9e-6 
    delta = 1e-7 

    Aggregation_Type = 'A_MPDU'   #A_MPDU or A_MSDU (HYBRID not fully supported)
    K_MSDU = 1 
    K_MPDU = k 
    L_MPDU_HEADER = 18 * 8 
    L_MSDU_HEADER = 14 * 8 
    if (k <= 1):
        Aggregation_Type = 'NONE' 

    N_DBPS = data_rate * T_SYMBOL_DATA   # number of data bits per OFDM symbol

    if (Aggregation_Type == 'NONE'):
        N_SYMBOLS = math.ceil((L_SERVICE + (L_MAC + L_DATA + L_APP_HDR) + L_TAIL)/N_DBPS)
        T_DATA = T_PHY_DATA + (T_SYMBOL_DATA * N_SYMBOLS)
        K_MPDU = 1
        K_MSDU = 1 

    if (Aggregation_Type == 'A_MSDU'):
        N_SYMBOLS = math.ceil((L_SERVICE + K_MPDU*(L_MAC + L_MPDU_HEADER + K_MSDU*(L_MSDU_HEADER + L_DATA + L_APP_HDR)) + L_TAIL)/N_DBPS)
        T_DATA = T_PHY_DATA + (T_SYMBOL_DATA * N_SYMBOLS)

    if (Aggregation_Type == 'A_MPDU'):
        N_SYMBOLS = math.ceil((L_SERVICE + K_MPDU*(L_MAC + L_MPDU_HEADER + L_DATA + L_APP_HDR) + L_TAIL)/N_DBPS)
        T_DATA = T_PHY_DATA + (T_SYMBOL_DATA * N_SYMBOLS)

    #Calculate ACK Duration
    N_DBPS = ack_rate * T_SYMBOL_ACK   # number of data bits per OFDM symbol
    N_SYMBOLS = math.ceil((L_SERVICE + L_ACK + L_TAIL)/N_DBPS)
    T_ACK = T_PHY_ACK + (T_SYMBOL_ACK * N_SYMBOLS)

    T_s = T_DATA + T_SIFS + T_ACK + T_DIFS 
    if difs == 1: #DIFS
        T_C = T_DATA + T_DIFS #+ T_SLOT
    else:
        T_s = T_DATA + T_SIFS + T_ACK + T_DIFS + delta 
        T_C = T_DATA + T_DIFS + T_SIFS + T_ACK + delta 

    T_S = T_s/(1-B) + T_SLOT 

    S_bianchi = np.zeros(len(nA)) 
    for j in range(len(nA)):
        n = nA[j]*1 
        W = CWmin + 1 
        m = math.log2((CWmax + 1)/(CWmin + 1)) 
        tau1 = np.linspace(0, 0.1, 100000)  
        p = 1 - np.power((1 - tau1),(n - 1))
        ps = p*0 

        for i in range(int(m)):
            ps = ps + np.power(2*p, i) 

        taup = 2./(1 + W + p*W*ps) 
        b = np.argmin(np.abs(tau1 - taup))
        tau = taup[b] 

        Ptr = 1 - math.pow((1 - tau), int(n))
        Ps = n*tau*math.pow((1 - tau), int(n-1))/Ptr

        S_bianchi[j] = K_MSDU*K_MPDU*Ps*Ptr*EP/((1-Ptr)*T_SLOT+Ptr*Ps*T_S+Ptr*(1-Ps)*T_C)/1e6 

    bianchi_result = S_bianchi
    return bianchi_result

## Generate All

In [ ]:
data_rates = [[8.603e6, 17.206e6, 25.8e6, 34.4e6, 51.5e6, 68.8e6, 77.4e6, 86e6, 103.2e6, 114.7e6, 129e6, 143.4e6], 
[17.2e6, 34.4e6, 51.5e6, 68.8e6, 103.2e6, 137.6e6, 154.9e6, 172.1e6, 206.5e6, 229.4e6, 258.1e6, 286.8e6],
[36e6, 72.1e6, 108.1e6, 144.1e6, 216.2e6, 288.2e6, 324.3e6, 360.3e6, 432.4e6, 480.4e6, 540.4e6, 600.5e6],
[72.1e6, 144.1e6, 216.2e6, 288.2e6, 432.4e6, 576.5e6, 648.5e6, 720.6e6, 864.7e6, 960.8e6, 1080.9e6, 1201e6]
]
ack_rates = [
    [6e6, 12e6, 12e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6],
    [6e6, 12e6, 12e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6],
    [6e6, 12e6, 12e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6],
    [6e6, 12e6, 12e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6, 24e6],
]
ack_rate  = 24e6
k_n = 1
difs = 1

tpt_adhoc = []
tpt_infra = []
error_adhoc = []
error_infra = []
for k in range(4):
    data_dir = "data_" + str(int(20*(math.pow(2,k))))
    tpt_ads = np.zeros(shape=(12,10))
    tpt_ins = np.zeros(shape=(12,10))
    error_ads = np.zeros(shape=(12,10))
    error_ins = np.zeros(shape=(12,10))

    for i in range(12):
        data_dir_ad = data_dir + "/mcs_a_"+ str(int(20*(math.pow(2,k)))) +"_" + str(i) + ".txt"
        data_dir_in = data_dir + "/mcs_infra_"+ str(int(20*(math.pow(2,k)))) +"_" + str(i) + ".txt"
        tpt_ad = np.array(data_analysis(data_dir_ad))
        tpt_in = np.array(data_analysis(data_dir_in))
        tpt_ads[i] = tpt_ad
        tpt_ins[i] = tpt_in

        bianchi_result = bianchi_ax(data_rates[k][i], ack_rates[k][i], k_n, difs)
        #errar_a = np.abs(tpt_ad - bianchi_result)/bianchi_result*100
        error_ad = (tpt_ad - bianchi_result)/bianchi_result*100
        error_in = (tpt_in - bianchi_result)/bianchi_result*100
        error_ads[i] = error_ad
        error_ins[i] = error_in
    tpt_adhoc.append(tpt_ads)
    tpt_infra.append(tpt_ins)
    error_adhoc.append(error_ads)
    error_infra.append(error_ins)
#     np.append(tpt_adhoc, tpt_ins)
#     np.append(tpt_infra, tpt_ads)
#     np.append(error_adhoc, error_ads)
#     np.append(error_infra, error_ins)

#np.savetxt('new.csv', tpt_adhoc[0], delimiter = '')  

In [ ]:
headers = ["HeMCS0","HeMCS1","HeMCS2","HeMCS3","HeMCS4","HeMCS5","HeMCS6","HeMCS7","HeMCS8","HeMCS9","HeMCS10","HeMCS11"]
for i in range(4):
    pf = pd.DataFrame(data=np.transpose(tpt_adhoc[i]))
    pf.to_csv("./addhoc_tpt.csv", encoding="utf-8-sig", header=headers, mode="a", index=False)
    pf = pd.DataFrame(data=np.transpose(tpt_infra[i]))
    pf.to_csv("./infra_tpt.csv", encoding="utf-8-sig", header=headers, mode="a", index=False)
    pf = pd.DataFrame(data=np.transpose(error_adhoc[i]))
    pf.to_csv("./addhoc_error.csv", encoding="utf-8-sig", header=headers, mode="a", index=False)
    pf = pd.DataFrame(data=np.transpose(error_infra[i]))
    pf.to_csv("./infra_error.csv", encoding="utf-8-sig", header=headers, mode="a", index=False)

In [ ]:
x =list(range(len(error_ad)))
total_width, n = 0.8, 2
width = total_width / n
 
plt.bar(x, error_ad, width=width, label='ad-hoc',fc = 'y')
for i in range(len(x)):
    x[i] = x[i] + width
plt.bar(x, error_in, width=width, label='infrastructure',fc = 'r')
plt.legend()
plt.show()

In [ ]:
error_adhoc

In [ ]:
error_infra

## 40MHZ

In [ ]:
tpt_ad_40 = []
tpt_infra_40 = []
error_adhoc = []
error_infra = []
for i in range(12):
    data_dir_ad = "data_40/mcs_a_40_"+ str(i)
    data_dir_infra = "data_40/mcs_"+ str(i) + "_infra_40.txt"
    tpt_ad = np.array(data_analysis(data_dir_ad))
    tpt_in = np.array(data_analysis(data_dir_infra))
    tpt_ad_40.append(tpt_ad)
    tpt_infra_40.append(tpt_in)
    bianchi_result = bianchi_ax(data_rates_40MHz[i], ack_rate, k, difs)
    #errar_a = np.abs(tpt_ad - bianchi_result)/bianchi_result*100
    error_ad = np.abs(tpt_ad - bianchi_result)/bianchi_result*100
    error_in = np.abs(tpt_in - bianchi_result)/bianchi_result*100
    error_adhoc.append(error_ad)
    error_infra.append(error_in)
    #print(i, errar_a)
    #print(tpt_ad)



In [ ]:
error_adhoc

In [ ]:
error_infra

In [ ]:
data_dir_infra = "data_40/mcs_"+ str(1) + "_infra_40.txt"

In [ ]:
tpt_in = np.array(data_analysis(data_dir_infra))

In [ ]:
tpt_in

In [ ]:
error_infra

In [ ]:
data_dir = "2mcs_3_in.txt"
tpt_in = np.array(data_analysis(data_dir))
bianchi_result = bianchi_ax(data_rates[0][2], 24e6, 2, 0, 1)
bianchi_result1 = bianchi_ax(data_rates[0][2], 24e6, 2, 1, 0)
bianchi_result2 = bianchi_ax(data_rates[0][2], 24e6, 2, 0, 0)
bianchi_result3 = bianchi_ax(data_rates[0][2], 24e6, 1, 1, 0)

In [ ]:
error_in = (tpt_in - bianchi_result)/bianchi_result*100

In [ ]:
error_in

In [ ]:
x = [5, 10 ,15, 20, 25, 30 ,35, 40 ,45, 50]
plt.plot(x,tpt_in,label='ns3',marker='*',color='blue', linestyle='-.',markersize=4)
plt.plot(x,bianchi_result,label='Bianchi',marker='x',color='red', linestyle='-',markersize=4)
plt.plot(x,bianchi_result1,label='Bianchi',marker='x',color='red', linestyle='-',markersize=4)
plt.plot(x,bianchi_result2,label='Bianchi',marker='x',color='red', linestyle='-',markersize=4)
plt.plot(x,bianchi_result3,label='Bianchi',marker='x',color='y', linestyle='-',markersize=4)
plt.xlabel('Node number')
plt.ylabel('Throughput (Mbps)')
plt.legend()
plt.savefig('mpdu2.png',dpi=300,format='png')
plt.show()

In [ ]:
data_dir = 'dis'
_tpt_5 = []
_tpt_6 = []
for i in range(5):
    data_dir_5 = data_dir + "/mcs5_band20_fre5_dis"+ str(int(30*i + 1))
    data_dir_6 = data_dir + "/mcs5_band20_fre6_dis"+ str(int(30*i + 1))
    tpt_5 = np.array(data_analysis(data_dir_5))
    tpt_6 = np.array(data_analysis(data_dir_6))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])


In [ ]:
_tpt_5

In [ ]:
_tpt_6

In [ ]:
x = [0, 30 ,60, 90, 120]
plt.plot(x,_tpt_5,label='5 GHz',marker='*',color='blue', linestyle='-.',markersize=4)
plt.plot(x,_tpt_6,label='6 GHz',marker='x',color='red', linestyle='-',markersize=4)
plt.xlabel('Distance')
plt.ylabel('Throughput (Mbps)')
plt.legend()
#plt.savefig('mpdu2.png',dpi=300,format='png')
plt.show()

In [ ]:
_tpt_7 = []
for i in range(8):
    data_dir_6 = data_dir + "/mcs5_band20_fre6_dis"+ str(int(10*i + 30))
    tpt_6 = np.array(data_analysis(data_dir_6))
    _tpt_7.append(tpt_6[0])

In [ ]:
plt.plot(_tpt_7)

In [ ]:
data_dir = 'mcs5'
_tpt_5 = []
_tpt_6 = []
for i in range(14):
    print(int(30*i + 10))
    data_dir_6 = data_dir + "/mcs5_band20_fre6_dis"+ str(int(30*i + 10))
    data_dir_5 = data_dir + "/mcs5_band20_fre5_dis"+ str(int(30*i + 10))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])
    
x = [10, 40 ,70, 100, 130, 160, 190, 220, 250, 280, 310, 340, 370,400]
plt.plot(x,_tpt_5,label='5 GHz',marker='*',color='blue', linestyle='-.',markersize=4)
plt.plot(x,_tpt_6,label='6 GHz',marker='x',color='red', linestyle='-',markersize=4)
plt.xlabel('Distance')
plt.ylabel('Throughput (Mbps)')
plt.legend()
#plt.savefig('mpdu2.png',dpi=300,format='png')
plt.show()

In [ ]:
x = [10, 40 ,70, 100, 130, 160, 190, 220, 250, 280, 310, 340, 370,400]
plt.plot(x,_tpt_5,label='5 GHz',marker='*',color='blue', linestyle='-.',markersize=4)
plt.plot(x,_tpt_6,label='6 GHz',marker='x',color='red', linestyle='-',markersize=4)
plt.xlabel('Distance')
plt.ylabel('Throughput (Mbps)')
plt.legend()
#plt.savefig('mpdu2.png',dpi=300,format='png')
plt.show()

In [ ]:
data_dir = 'mcs5'
_tpt_5 = []
_tpt_6 = []
for i in range(2):
    print(int(30*i + 10))
    data_dir_6 = data_dir + "/mcs5_band20_fre6_dis"+ str(int(30*i + 10))
    data_dir_5 = data_dir + "/mcs5_band20_fre5_dis"+ str(int(30*i + 10))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])

for i in range(5):
    print(int(5*i + 40))
    data_dir_6 = data_dir + "/mcs5_band20_fre6_dis"+ str(int(5*i + 45))
    data_dir_5 = data_dir + "/mcs5_band20_fre5_dis"+ str(int(5*i + 45))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])

for i in range(12):
    print(int(30*i + 100))
    data_dir_6 = data_dir + "/mcs5_band20_fre6_dis"+ str(int(30*i + 70))
    data_dir_5 = data_dir + "/mcs5_band20_fre5_dis"+ str(int(30*i + 70))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])
    
x = [10, 40,45, 50, 55, 60, 65, 70, 100, 130, 160, 190, 220, 250, 280, 310, 340, 370,400]
plt.plot(x,_tpt_5,label='5 GHz',marker='*',color='blue', linestyle='-.',markersize=4)
plt.plot(x,_tpt_6,label='6 GHz',marker='x',color='red', linestyle='-',markersize=4)
plt.xlabel('Distance')
plt.ylabel('Throughput (Mbps)')
plt.title('HeMcs=5, Channel Bandwidth = 20 MHz')
plt.legend()
plt.savefig('mcs5_20.png',dpi=300,format='png')
plt.show()

In [ ]:
data_dir = 'mcs5'
_tpt_5 = []
_tpt_6 = []
for i in range(2):
    print(int(30*i + 10))
    data_dir_6 = data_dir + "/mcs5_band40_fre6_dis"+ str(int(30*i + 10))
    data_dir_5 = data_dir + "/mcs5_band40_fre5_dis"+ str(int(30*i + 10))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])

for i in range(5):
    print(int(5*i + 40))
    data_dir_6 = data_dir + "/mcs5_band40_fre6_dis"+ str(int(5*i + 45))
    data_dir_5 = data_dir + "/mcs5_band40_fre5_dis"+ str(int(5*i + 45))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])

for i in range(12):
    print(int(30*i + 100))
    data_dir_6 = data_dir + "/mcs5_band40_fre6_dis"+ str(int(30*i + 70))
    data_dir_5 = data_dir + "/mcs5_band40_fre5_dis"+ str(int(30*i + 70))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])
    
x = [10, 40,45, 50, 55, 60, 65, 70, 100, 130, 160, 190, 220, 250, 280, 310, 340, 370,400]
plt.plot(x,_tpt_5,label='5 GHz',marker='*',color='blue', linestyle='-.',markersize=4)
plt.plot(x,_tpt_6,label='6 GHz',marker='x',color='red', linestyle='-',markersize=4)
plt.xlabel('Distance')
plt.ylabel('Throughput (Mbps)')
plt.legend()
plt.title('HeMcs=5, Channel Bandwidth = 40 MHz')
plt.legend()
plt.savefig('mcs5_40.png',dpi=300,format='png')
plt.show()

In [ ]:
data_dir = 'mcs5'
_tpt_5 = []
_tpt_6 = []
for i in range(2):
    print(int(30*i + 10))
    data_dir_6 = data_dir + "/mcs5_band80_fre6_dis"+ str(int(30*i + 10))
    data_dir_5 = data_dir + "/mcs5_band80_fre5_dis"+ str(int(30*i + 10))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])

for i in range(5):
    print(int(5*i + 40))
    data_dir_6 = data_dir + "/mcs5_band80_fre6_dis"+ str(int(5*i + 45))
    data_dir_5 = data_dir + "/mcs5_band80_fre5_dis"+ str(int(5*i + 45))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])

for i in range(12):
    print(int(30*i + 100))
    data_dir_6 = data_dir + "/mcs5_band80_fre6_dis"+ str(int(30*i + 70))
    data_dir_5 = data_dir + "/mcs5_band80_fre5_dis"+ str(int(30*i + 70))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])
    
x = [10, 40,45, 50, 55, 60, 65, 70, 100, 130, 160, 190, 220, 250, 280, 310, 340, 370,400]
plt.plot(x,_tpt_5,label='5 GHz',marker='*',color='blue', linestyle='-.',markersize=4)
plt.plot(x,_tpt_6,label='6 GHz',marker='x',color='red', linestyle='-',markersize=4)
plt.xlabel('Distance')
plt.ylabel('Throughput (Mbps)')
plt.title('HeMcs=5, Channel Bandwidth = 80 MHz')
plt.legend()
plt.savefig('mcs5_80.png',dpi=300,format='png')
plt.show()

In [ ]:
data_dir = 'mcs5'
_tpt_5 = []
_tpt_6 = []
for i in range(2):
    print(int(30*i + 10))
    data_dir_6 = data_dir + "/mcs5_band160_fre6_dis"+ str(int(30*i + 10))
    data_dir_5 = data_dir + "/mcs5_band160_fre5_dis"+ str(int(30*i + 10))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])

for i in range(5):
    print(int(5*i + 40))
    data_dir_6 = data_dir + "/mcs5_band160_fre6_dis"+ str(int(5*i + 45))
    data_dir_5 = data_dir + "/mcs5_band160_fre5_dis"+ str(int(5*i + 45))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])

for i in range(12):
    print(int(30*i + 100))
    data_dir_6 = data_dir + "/mcs5_band160_fre6_dis"+ str(int(30*i + 70))
    data_dir_5 = data_dir + "/mcs5_band160_fre5_dis"+ str(int(30*i + 70))
    tpt_6 = np.array(data_analysis(data_dir_6))
    tpt_5 = np.array(data_analysis(data_dir_5))
    _tpt_5.append(tpt_5[0])
    _tpt_6.append(tpt_6[0])
    
x = [10, 40,45, 50, 55, 60, 65, 70, 100, 130, 160, 190, 220, 250, 280, 310, 340, 370,400]
plt.plot(x, _tpt_5,label='5 GHz',marker='*',color='blue', linestyle='-.',markersize=4)
plt.plot(x, _tpt_6,label='6 GHz',marker='x',color='red', linestyle='-',markersize=4)
plt.xlabel('Distance')
plt.ylabel('Throughput (Mbps)')
plt.title('HeMcs=5, Channel Bandwidth = 160 MHz')
plt.legend()
plt.savefig('mcs5_160.png',dpi=300,format='png')
plt.show()

In [ ]:
ack_rate  = 24e6
k = 0
difs = 1
useExplicitBar = 0
data_rate = 960.8e6
nA = np.linspace(5, 50, 10)
CWmin = 15 
CWmax = 1023 
L_DATA = 1500 * 8 # data size in bits
L_ACK = 14 * 8    # ACK size in bits
B = 1/(CWmin+1) 
B=0 
EP = L_DATA/(1-B)
T_GI = 800e-9                  # guard interval in seconds
T_SYMBOL_ACK = 4e-6            # symbol duration in seconds (for ACK)
T_SYMBOL_DATA = 12.8e-6 + T_GI # symbol duration in seconds (for DATA)
T_PHY_ACK = 20e-6              # PHY preamble & header duration in seconds (for ACK)
T_PHY_DATA = 44e-6             # PHY preamble & header duration in seconds (for DATA)
L_SERVICE = 16                 # service field length in bits
L_TAIL = 6                     # tail lengthh in bits
L_MAC = (30) * 8               # MAC header size in bits
L_APP_HDR = 8 * 8              # bits added by the upper layer(s)
T_SIFS = 16e-6 
T_DIFS = 34e-6 
T_SLOT = 9e-6 
delta = 1e-7 
L_BAR = 24 * 8
Aggregation_Type = 'A_MPDU'   #A_MPDU or A_MSDU (HYBRID not fully supported)
K_MSDU = 1 
K_MPDU = k 
L_MPDU_HEADER = 18 * 8 
L_MSDU_HEADER = 14 * 8 
if (k == 0):
    Aggregation_Type = 'NONE' 

N_DBPS = data_rate * T_SYMBOL_DATA   # number of data bits per OFDM symbol

if (Aggregation_Type == 'NONE'):
    N_SYMBOLS = math.ceil((L_SERVICE + (L_MAC + L_DATA + L_APP_HDR) + L_TAIL)/N_DBPS)
    T_DATA = T_PHY_DATA + (T_SYMBOL_DATA * N_SYMBOLS)
    K_MPDU = 1
    K_MSDU = 1 

if (Aggregation_Type == 'A_MSDU'):
    N_SYMBOLS = math.ceil((L_SERVICE + K_MPDU*(L_MAC + L_MPDU_HEADER + K_MSDU*(L_MSDU_HEADER + L_DATA + L_APP_HDR)) + L_TAIL)/N_DBPS)
    T_DATA = T_PHY_DATA + (T_SYMBOL_DATA * N_SYMBOLS)

if (Aggregation_Type == 'A_MPDU'):
    N_SYMBOLS = math.ceil((L_SERVICE + K_MPDU*(L_MAC + L_MPDU_HEADER + L_DATA + L_APP_HDR) + L_TAIL)/N_DBPS)
    T_DATA = T_PHY_DATA + (T_SYMBOL_DATA * N_SYMBOLS)

#Calculate ACK Duration
N_DBPS = ack_rate * T_SYMBOL_ACK   # number of data bits per OFDM symbol
N_SYMBOLS = math.ceil((L_SERVICE + L_ACK + L_TAIL)/N_DBPS)
T_ACK = T_PHY_ACK + (T_SYMBOL_ACK * N_SYMBOLS)

T_s = T_DATA + T_SIFS + T_ACK + T_DIFS 
if difs == 1: #DIFS
    T_C = T_DATA + T_DIFS 
else:
    T_s = T_DATA + T_SIFS + T_ACK + T_DIFS + delta 
    T_C = T_DATA + T_DIFS + T_SIFS + T_ACK + delta 

if useExplicitBar == 1:
    T_BAR = (math.ceil((L_SERVICE + L_BAR + L_TAIL)/N_DBPS))*T_SYMBOL_ACK + T_PHY_ACK
    T_BACKOFF = CWmin * T_SLOT/2 
    T_C = T_DATA + T_DIFS + T_SIFS + T_ACK + delta + T_BACKOFF + T_BAR + T_DIFS
T_S = T_s/(1-B) + T_SLOT 

S_bianchi = np.zeros(len(nA)) 
for j in range(len(nA)):
    n = nA[j]*1 
    W = CWmin + 1 
    m = math.log2((CWmax + 1)/(CWmin + 1)) 
    tau1 = np.linspace(0, 0.1, 100000)  
    p = 1 - np.power((1 - tau1),(n - 1))
    ps = p*0 

    for i in range(int(m)):
        ps = ps + np.power(2*p, i) 

    taup = 2./(1 + W + p*W*ps) 
    b = np.argmin(np.abs(tau1 - taup))
    tau = taup[b] 

    Ptr = 1 - math.pow((1 - tau), int(n))
    Ps = n*tau*math.pow((1 - tau), int(n-1))/Ptr

    S_bianchi[j] = K_MSDU*K_MPDU*Ps*Ptr*EP/((1-Ptr)*T_SLOT+Ptr*Ps*T_S+Ptr*(1-Ps)*T_C)/1e6 

bianchi_result = S_bianchi

In [ ]:
T_ACK

In [ ]:
T_DATA

In [ ]:
T_s

In [ ]:
data_rates = [[8.603e6, 17.206e6, 25.8e6, 34.4e6, 51.5e6, 68.8e6, 77.4e6, 86e6, 103.2e6, 114.7e6, 129e6, 143.4e6], 
[17.2e6, 34.4e6, 51.5e6, 68.8e6, 103.2e6, 137.6e6, 154.9e6, 172.1e6, 206.5e6, 229.4e6, 258.1e6, 286.8e6],
[36e6, 72.1e6, 108.1e6, 144.1e6, 216.2e6, 288.2e6, 324.3e6, 360.3e6, 432.4e6, 480.4e6, 540.4e6, 600.5e6],
[72.1e6, 144.1e6, 216.2e6, 288.2e6, 432.4e6, 576.5e6, 548.5e6, 720.6e6, 864.7e6, 960.8e6, 1080.9e6, 1201e6]
]
ack_rate  = 24e6
k_n = 1
difs = 1

tpt_adhoc = []
tpt_infra = []
error_adhoc = []
error_infra = []
for k in range(4):
    data_dir = "data_" + str(int(20*(math.pow(2,k))))
    i = 2
    data_dir_ad = data_dir + "/mcs_a_"+ str(int(20*(math.pow(2,k)))) +"_" + str(i) + ".txt"
    data_dir_in = data_dir + "/mcs_infra_"+ str(int(20*(math.pow(2,k)))) +"_" + str(i) + ".txt"
    tpt_ad = np.array(data_analysis(data_dir_ad))
    tpt_in = np.array(data_analysis(data_dir_in))
    tpt_ads = tpt_ad
    tpt_ins = tpt_in
    bianchi_result = bianchi_ax(data_rates[k][2], 12e6, 1, difs)
    #errar_a = np.abs(tpt_ad - bianchi_result)/bianchi_result*100
    error_ad = (tpt_ad - bianchi_result)/bianchi_result*100
    error_in = (tpt_in - bianchi_result)/bianchi_result*100
    error_ads = error_ad
    error_ins = error_in
    
    
    tpt_adhoc.append(tpt_ads)
    tpt_infra.append(tpt_ins)
    error_adhoc.append(error_ads)
    error_infra.append(error_ins)
#     np.append(tpt_adhoc, tpt_ins)
#     np.append(tpt_infra, tpt_ads)
#     np.append(error_adhoc, error_ads)
#     np.append(error_infra, error_ins)

#np.savetxt('new.csv', tpt_adhoc[0], delimiter = '')  

In [ ]:
error_adhoc

In [ ]:
error_infra